# bert-huggingface
## 教學目標
利用 Hugging Face 套件快速使用 BERT 模型來進行下游任務訓練
- 單一句型分類任務 (single sentence text classification)
- 成對句型分類任務 (sentence pair text classification)
- 問答任務 (question answering)
- 命名實體辨識 (named-entity recognition / token-level text classification)

# 單一句型分類任務
## 下載資料
我們使用 IMDb reviews 資料集作為範例

In [1]:
!pip install wget
import wget
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = wget.download(url, out='./')


In [3]:
import tarfile

# 指定檔案位置，並解壓縮 .gz 結尾的壓縮檔
tar = tarfile.open('aclImdb_v1.tar.gz', 'r:gz')
tar.extractall()

## 接下來我們要進行資料前處理
但首先要觀察解壓縮後的資料夾結構
```
aclImdb---
        |--train
        |    |--neg
        |    |--pos
        |    |--...
        |--test
        |    |--neg
        |    |--pos
        |    |--...
        |--imdb.vocab
        |--imdbEr.text
        |--README
```
其中train和test資料夾中分別又有neg和pos兩種資料夾

我們要針對這兩個目標資料夾進行處理

In [19]:
# 載入 pathlib 模組 (Python3.4+)
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        # 利用iterdir() 來列出資料夾底下的所有檔案，此功能等同於 os.path.listdir()
#         for text_file in (split_dir/label_dir).iterdir():
        # 若知道副檔名為.txt
        for text_file in (split_dir/label_dir).glob("*.txt"):
            tmp_text = text_file.read_text()
#             print(tmp_text)
#             exit()
            texts.append(tmp_text)
            labels.append(0 if label_dir == "neg" else 1)
    
    return texts, labels

In [ ]:
train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')